## CNN sentence classifier - Round 2 of Hyperparameter tuning

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 12.2 ms, sys: 12.1 ms, total: 24.3 ms
Wall time: 23.3 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31 s, sys: 159 ms, total: 31.1 s
Wall time: 30.8 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:

# this is very quick, so we can use grid search on these models.

run_logfile = "runs2bal.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal",
                      model_dir        = "../models/CNNBal")
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 8ms/step - loss: 0.6485 - accuracy: 0.6275 - val_loss: 0.5402 - val_accuracy: 0.7301
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-102732/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.4376 - accuracy: 0.8002 - val_loss: 0.4102 - val_accuracy: 0.8201
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-102732/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.2926 - accuracy: 0.8765 - val_loss: 0.2854 - val_accuracy: 0.8906
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-102732/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1780 - accuracy: 0.9357 - val_loss: 0.2139 - val_accuracy: 0.9268
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-102732/assets
Epoch 5/20
288/288 [=============

Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0890 - accuracy: 0.9699 - val_loss: 0.2589 - val_accuracy: 0.9191
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0665 - accuracy: 0.9783 - val_loss: 0.2477 - val_accuracy: 0.9268
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0429 - accuracy: 0.9863 - val_loss: 0.2265 - val_accuracy: 0.9344
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-102846/assets
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0357 - accuracy: 0.9889 - val_loss: 0.1902 - val_accuracy: 0.9498
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-102846/assets
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0262 - accuracy: 0.9917 - val_loss: 0.2975 - val_accuracy: 0.9281
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0301 - accuracy: 0.9900 - val_loss: 0.2918 - val_accuracy: 0.9326
E

Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0171 - accuracy: 0.9953 - val_loss: 0.3171 - val_accuracy: 0.9349
Epoch 13/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0134 - accuracy: 0.9961 - val_loss: 0.3749 - val_accuracy: 0.9340
Epoch 14/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0076 - accuracy: 0.9984 - val_loss: 0.5386 - val_accuracy: 0.9091
Epoch 15/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0132 - accuracy: 0.9957 - val_loss: 0.5436 - val_accuracy: 0.9141
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0117 - accuracy: 0.9960 - val_loss: 0.3893 - val_accuracy: 0.9326
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0084 - accuracy: 0.9972 - val_loss: 0.4659 - val_accuracy: 0.9317
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0098 - accuracy: 0.9975 - val_loss: 0.5810 - val_accuracy: 0.9141

288/288 [==============================] - 2s 5ms/step - loss: 0.0131 - accuracy: 0.9963 - val_loss: 0.3424 - val_accuracy: 0.9435
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0106 - accuracy: 0.9965 - val_loss: 0.4397 - val_accuracy: 0.9159
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0134 - accuracy: 0.9957 - val_loss: 0.5332 - val_accuracy: 0.9146



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6289 - accuracy: 0.6371 - val_loss: 0.4152 - val_accuracy: 0.8246
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-103146/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.3966 - accuracy: 0.8243 - val_loss: 0.4481 - val_accuracy: 0.8070
Epoch 3/20
288/288 [==================

INFO:tensorflow:Assets written to: ../models/CNNBal/210409-103258/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1219 - accuracy: 0.9583 - val_loss: 0.3253 - val_accuracy: 0.8901
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0670 - accuracy: 0.9795 - val_loss: 0.2612 - val_accuracy: 0.9132
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0436 - accuracy: 0.9874 - val_loss: 0.2725 - val_accuracy: 0.9241
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-103258/assets
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0314 - accuracy: 0.9914 - val_loss: 0.3425 - val_accuracy: 0.9123
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0249 - accuracy: 0.9926 - val_loss: 0.3415 - val_accuracy: 0.9159
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0262 - accuracy: 0.9917 - val_loss: 0.3113 - val_accuracy: 0.9331
IN

...

288/288 [==============================] - 2s 7ms/step - loss: 0.0111 - accuracy: 0.9965 - val_loss: 0.3714 - val_accuracy: 0.9236
Epoch 14/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0094 - accuracy: 0.9971 - val_loss: 0.4592 - val_accuracy: 0.9218
Epoch 15/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0067 - accuracy: 0.9978 - val_loss: 0.3234 - val_accuracy: 0.9399
Epoch 16/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 0.6942 - val_accuracy: 0.8825
Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0116 - accuracy: 0.9970 - val_loss: 0.4328 - val_accuracy: 0.9159
Epoch 18/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.3103 - val_accuracy: 0.9489
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0057 - accuracy: 0.9978 - val_loss: 0.4121 - val_accuracy: 0.9290
Epoch 20/20




##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 128
Adding Convolution: Kernel Size: 8, Filter Count: 128
Adding Convolution: Kernel Size: 32, Filter Count: 128
Adding Convolution: Kernel Size: 64, Filter Count: 128
Epoch 1/20
288/288 [==============================] - 3s 9ms/step - loss: 0.6120 - accuracy: 0.6734 - val_loss: 0.4918 - val_accuracy: 0.7500
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-120321/assets
Epoch 2/20
288/288 [==============================] - 2s 7ms/step - loss: 0.3167 - accuracy: 0.8658 - val_loss: 0.2791 - val_accuracy: 0.8811
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-120321/assets
Epoch 3/20
288/288 [==============================] - 2s 7ms/step - loss: 0.1506 - accuracy: 0.9474 - val_loss: 0.2249 - val_accuracy: 0.9109
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-120321/assets
Epoch 4/20
288/288 [==============================] - 2s

Epoch 4/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0571 - accuracy: 0.9826 - val_loss: 0.2571 - val_accuracy: 0.9250
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-120459/assets
Epoch 5/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0308 - accuracy: 0.9907 - val_loss: 0.2004 - val_accuracy: 0.9476
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-120459/assets
Epoch 6/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0206 - accuracy: 0.9941 - val_loss: 0.3684 - val_accuracy: 0.9213
Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0214 - accuracy: 0.9920 - val_loss: 0.2957 - val_accuracy: 0.9363
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0139 - accuracy: 0.9957 - val_loss: 0.5853 - val_accuracy: 0.8865
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0128 - accuracy: 0.9961 - val_loss: 0.2506 - val_accuracy: 0.9458
Ep

Epoch 10/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0155 - accuracy: 0.9947 - val_loss: 0.2981 - val_accuracy: 0.9331
Epoch 11/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0131 - accuracy: 0.9959 - val_loss: 0.3525 - val_accuracy: 0.9200
Epoch 12/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0089 - accuracy: 0.9977 - val_loss: 0.2093 - val_accuracy: 0.9566
INFO:tensorflow:Assets written to: ../models/CNNBal/210409-120633/assets
Epoch 13/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0097 - accuracy: 0.9969 - val_loss: 0.3598 - val_accuracy: 0.9344
Epoch 14/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0122 - accuracy: 0.9971 - val_loss: 0.4608 - val_accuracy: 0.9182
Epoch 15/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.3622 - val_accuracy: 0.9299
Epoch 16/20
288/288 [==============================] - 2s 7ms/step - 

288/288 [==============================] - 2s 7ms/step - loss: 0.0113 - accuracy: 0.9956 - val_loss: 0.6312 - val_accuracy: 0.8888
Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0131 - accuracy: 0.9963 - val_loss: 0.3130 - val_accuracy: 0.9372
Epoch 18/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0099 - accuracy: 0.9961 - val_loss: 0.3102 - val_accuracy: 0.9403
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0077 - accuracy: 0.9979 - val_loss: 0.5128 - val_accuracy: 0.9200
Epoch 20/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0090 - accuracy: 0.9969 - val_loss: 0.5791 - val_accuracy: 0.9105



##########################################################################









### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/